The goal here is to show how we can implement Euler's method using Julia! 
Let's consider a disability insurance, this means that $S = \{*, \diamond, \dagger\}$, we want to find the survivalprobabilities

$$\begin{aligned}
P(t,s) &= 
\begin{bmatrix}
    p_{**}(t,s)         & p_{*\diamond}(t,s)        & p_{*\dagger}(t,s) \\
    p_{\diamond*}(t,s)  & p_{\diamond\diamond}(t,s) & p_{\diamond\dagger}(t,s) \\
    0                   &  0                        & 1
\end{bmatrix}
\end{aligned}$$

In [2]:
using LinearAlgebra

In [4]:
#states: 
# 0: alive, 1: disabeld, 2: deceased

# Transition rate matrix Λ
function Λ(t)
    #state0:
    μ01(t) = 0.0004 + 10^(0.06*t-5.46)
    μ02(t) = 0.0005 + 10^(0.038*t-4.12)
    μ00(t) = -(μ01(t) + μ02(t))
    #state1:
    μ10(t) = 0.05 
    μ12(t) = μ02(t)
    μ11(t) = -(μ10(t)+μ12(t))
    #state2:
    # transition rates in the deceased state are zero
    
    L = [μ00(t) μ01(t) μ02(t)
         μ10(t) μ11(t) μ12(t)
         0       0     0    ]
    
    return L
end


Λ (generic function with 1 method)

In [5]:
function f(t,M)
    return M*Λ(t)
end

function Euler(t0, n_states, h, tn)
    """
    (t0): Int initial age 
    (n_states): Int, number of states
    (h): Float, stepsize 
    (tn): Int, final age
    """
    P0 = Matrix(1.0*I, n_states,n_states)

    if t0 == tn
        return P0
    end

    N = Int(round((tn-t0)/h))
    D = Int(size(P0)[1])
    # store N+1 (DxD)-matricies
    P = zeros(Float64,D,D,N+1) 
    P[:,:,1] = P0 

    
    for n in 1:N
        P[:,:, n+1] = P[:,:,n] + h*f(t0+n*h, P[:,:,n])
    end

    return P
end 

Euler (generic function with 1 method)

In [6]:
t0 = 30 
n_states = 3
h = 1/12     # stepsize 
tn = 120

sol = Euler(t0, n_states, h, tn) # transition probabilities from 30 to 120 with stepsize h =1/12
print(sol[1 , 1, 1:10])          # first 10 steps of survival


[1.0, 0.9998186582432559, 0.9996367084438109, 0.9994541437695489, 0.9992709573419496, 0.9990871422356756, 0.998902691478156, 0.9987175980491666, 0.9985318548804079, 0.998345454855079]

RungeKutta method

In [9]:
function k1(t,M)
    return f(t,M)
end

function k2(t,M)
    return f(t+h/2, M +h*k1(t,M)/2)
end

function k3(t,M)
    return f(t+h/2, M+ h*k2(t, M)/2)
end 

function k4(t,M)
    return f(t+h, M + h*k3(t,M))
end


k4 (generic function with 1 method)

In [10]:
function RK4(t0, n_states, h, tn)
    P0 = Matrix(1.0*I, n_states,n_states)

    if t0 == P0 
        return P0
    end 

    N = Int(round((tn-t0)/h))
    D = Int(size(P0)[1])
    # store N+1 (DxD)-matricies
    P = zeros(Float64,D,D,N+1) 
    P[:,:,1] = P0 

    for n in 1:N
        P[:,:,n+1] = P[:,:,n] + (h/6)*(k1(t0 + n*h, P[:,:,n]) + 2*k2(t0 +n*h, P[:,:,n]) +
                                       2*k3(t0 +n*h, P[:,:, n]) + k4(t0 +n*h, P[:,:,n]))
    end

    return P
end 
    

RK4 (generic function with 1 method)

In [11]:
sol_RK = RK4(t0, n_states, h, tn)
print(sol_RK[1,1,1:10]) 

[1.0, 0.999818354796958, 0.9996360981583859, 0.9994532232351914, 0.9992697231316997, 0.9990855909052406, 0.9989008195657305, 0.9987154020752523, 0.9985293313476322, 0.9983426002480129]